In [ ]:
# todo: time series
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
colum = ['tpep_pickup_datetime'] # scelgo la colonna tpep_pickup_datetime
df_2015_01_filtered = pd.read_csv(file_path1, usecols=colum)    # Load the dataset  in a DataFrame
df_2016_01 = pd.read_csv(file_path2, usecols=colum)     
df_2016_02 = pd.read_csv(file_path3, usecols=colum)
df_2016_03 = pd.read_csv(file_path4, usecols=colum)

df_total = pd.concat([df_2015_01_filtered, df_2016_01, df_2016_02, df_2016_03], ignore_index=True)

df_total = pd.concat([df_2015_01_filtered, df_2016_01, df_2016_02, df_2016_03], ignore_index=True)

# convert the column to datetime
df_total['tpep_pickup_datetime'] = pd.to_datetime(df_total['tpep_pickup_datetime'])

# Calcolare la media e la mediana delle ore
mu_decimal = df_total['hour'].mean()
median_hour = df_total['hour'].median()
sigma = df_total['hour'].std()

# Convertire la media decimale in formato HH:MM
hours = int(mu_decimal)
minutes = int((mu_decimal - hours) * 60)

# Stampare media e mediana
print(f"La media delle ore è: {hours}:{minutes:02d}")
print(f"La mediana delle ore è: {int(median_hour)}:00")

# Ordinare i valori per distanza dalla media
df_total['distance_from_mean'] = np.abs(df_total['hour'] - mu_decimal)
df_sorted = df_total.sort_values('distance_from_mean')

# Plot della distribuzione con media e mediana
xmin, xmax = 0, 23
x = np.linspace(xmin, xmax, 100)
gaussian_curve = norm.pdf(x, mu_decimal, sigma)

plt.figure(figsize=(10, 6))
plt.plot(x, gaussian_curve, 'r-', lw=2, label='Adattamento Gaussiano')  # Adattamento Gaussiano

# Creare l'istogramma con binwidth=1 per ogni ora
sns.histplot(df_sorted['hour'], binwidth=1, color='skyblue', edgecolor='black', stat='density', kde=False, alpha=0.6)

# Linee per la media, mediana e deviazione standard
plt.axvline(mu_decimal, color='red', linestyle='--', label=f'Media ({hours}:{minutes:02d})')
plt.axvline(mu_decimal + sigma, color='green', linestyle='--', label='Media + 1σ')
plt.axvline(mu_decimal - sigma, color='green', linestyle='--', label='Media - 1σ')

# Titoli e legende
plt.title("Distribuzione delle corse per ora con Media e Mediana")
plt.xlabel("Ora")
plt.ylabel("Densità")
plt.xticks(np.arange(0, 24, 1))
plt.xlim(xmin, xmax)  # Imposta il limite dell'asse x per centrarlo correttamente
plt.legend()
plt.show()

# Visualizzazione outliers con boxplot
plt.figure(figsize=(10, 2))
sns.boxplot(x=df_total['hour'], color='skyblue')
plt.title("Distribuzione dei valori di Hour con Outliers")
plt.xlabel("Ora")
plt.xlim(xmin, xmax)  # Imposta il limite dell'asse x per centrarlo correttamente nel boxplot
plt.show()
